# Code Cypher Pipeline

In [1]:
# libraries used to extract, clean and manipulate the data
from helpers import *
import pandas as pd
import numpy as np
import string

# to plot the graphs
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn')

# library used to count the frequency of words
from sklearn.feature_extraction.text import CountVectorizer

# to create the sentiment analysis model, tokenization and lemmatization
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import word_tokenize
import nltk.data
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('vader_lexicon')
nltk.download('punkt')

# genius token
client_id = 'PxzS3GRJYkyH8MVn1sgwMAzAiUSlqd_BICSisAVOlCCW9AmmzDg0IPCJC_qd1_qq'
client_secret = 'lTdOrNvmw0AoKElNZru1QDPl55sFy3EuoyXtWf4z9SII0Z7LhDcSwqRFU7AOSStuHG6CetaD2p8V_UFSooJJ8Q'
client_access = 'REZh59o8DCDIAX0B66n5eiKSm9ZGzbcsQ5WVywPdswHmRGRTgJOf-wIvZ0aFeK44'

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/vivekbhupatiraju/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/vivekbhupatiraju/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import text2emotion as te

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vivekbhupatiraju/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/vivekbhupatiraju/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/vivekbhupatiraju/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
df0 = search_data('Kanye West', ['On Sight', 'Blood on the Leaves', 'Heartless', 'Love Lockdown', 'Touch the Sky'], 25, client_access)
df0

Searching for songs by Kanye West...

Song 1: "Mercy"
Song 2: "Father Stretch My Hands, Pt. 1"
Song 3: "Monster"

Reached user-specified song limit (3).
Done. Found 3 songs.
Searching for "On Sight" by Kanye West...
Done.
Searching for "Blood on the Leaves" by Kanye West...
Done.
Searching for "Heartless" by Kanye West...
Done.
Searching for "Love Lockdown" by Kanye West...
Done.
Searching for "Touch the Sky" by Kanye West...
Done.


,artist,title,lyric
0,Kanye West,On Sight,"[Produced by Benji B, Daft Punk, Kanye West & ..."
1,Kanye West,Blood on the Leaves,"[Produced by TNGHT, Kanye West, Mike Dean, Arc..."
2,Kanye West,Heartless,"[Chorus]\nIn the night, I hear 'em talk\nThe c..."
3,Kanye West,Love Lockdown,"[Verse 1]\nI'm not loving you, way I wanted to..."
4,Kanye West,Touch the Sky,[Chorus 1: Kanye West]\nI gotta testify\nCome ...


In [4]:
df = clean_lyrics(df0,'lyric')
df.to_csv('lyrics.csv',index=False)
df.head(10)

/Users/vivekbhupatiraju/Dropbox/Code Projects/code-cypher-2021/helpers.py:52: FutureWarning: The default value of regex will change from True to False in a future version.
  df[column] = df[column].str.replace(r"verse |[1|2|3]|chorus|bridge|outro","").str.replace("[","").str.replace("]","")
/Users/vivekbhupatiraju/Dropbox/Code Projects/code-cypher-2021/helpers.py:52: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[column] = df[column].str.replace(r"verse |[1|2|3]|chorus|bridge|outro","").str.replace("[","").str.replace("]","")
/Users/vivekbhupatiraju/Dropbox/Code Projects/code-cypher-2021/helpers.py:53: FutureWarning: The default value of regex will change from True to False in a future version.
  df[column] = df[column].str.lower().str.replace(r"instrumental|intro|guitar|solo","")
/Users/vivekbhupatiraju/Dropbox/Code Projects/co

,artist,title,lyric
0,Kanye West,On Sight,produced by benji b daft punk kanye west mike...
1,Kanye West,Blood on the Leaves,produced by tnght kanye west mike dean arca ca...
2,Kanye West,Heartless,in the night i hear 'em talk the coldest story...
3,Kanye West,Love Lockdown,i'm not loving you way i wanted to what i had ...
4,Kanye West,Touch the Sky,kanye west i gotta testify come up in the spot...


## Sentiment Analysis

In [7]:
# Create lists to store the different scores for each word
negative = []
neutral = []
positive = []
compound = []

# Initialize the model
sid = SentimentIntensityAnalyzer()

# Iterate for each row of lyrics and append the scores
for i in df.index:
  scores = sid.polarity_scores(df['lyric'].iloc[i])
  negative.append(scores['neg'])
  neutral.append(scores['neu'])
  positive.append(scores['pos'])
  compound.append(scores['compound'])

# Create 4 columns to the main data frame  for each score
df['negative'] = negative
df['neutral'] = neutral
df['positive'] = positive
df['compound'] = compound
df.head()

,artist,title,lyric,negative,neutral,positive,compound
0,Kanye West,Mercy,fuzzy jones well it is a weepin' and a moanin'...,0.072,0.834,0.094,0.9683
1,Kanye West,"Father Stretch My Hands, Pt. 1",pastor tl barrett choir future you're the on...,0.074,0.835,0.091,0.9152
2,Kanye West,Monster,justin vernon i shoot the lights out hide 'til...,0.157,0.739,0.104,-0.9968
3,Kanye West,Ultralight Beam,samoria green natalie green yes god we don't ...,0.091,0.645,0.265,0.9991
4,Kanye West,Bound 2,ponderosa twins plus one brenda lee bbbbbound...,0.192,0.689,0.119,-0.9925


In [7]:
df

,artist,title,lyric,Angry,Fear,Happy,Sad,Surprise
0,Kanye West,On Sight,produced by benji b daft punk kanye west mike...,0.03,0.44,0.13,0.21,0.21
1,Kanye West,Blood on the Leaves,produced by tnght kanye west mike dean arca ca...,0.00,0.29,0.11,0.29,0.32
2,Kanye West,Heartless,in the night i hear 'em talk the coldest story...,0.04,0.12,0.04,0.51,0.28
3,Kanye West,Love Lockdown,i'm not loving you way i wanted to what i had ...,0.03,0.13,0.39,0.39,0.06
4,Kanye West,Touch the Sky,kanye west i gotta testify come up in the spot...,0.02,0.25,0.12,0.27,0.34


In [11]:
emotions = ['angry', 'sad', 'fear', 'surprise', 'happy']
song_data = {e: [] for e in emotions}

for i in df.index:
  scores = te.get_emotion(df['lyric'].iloc[i])
  print(df['title'][i], scores)

On Sight {'Happy': 0.13, 'Angry': 0.03, 'Surprise': 0.21, 'Sad': 0.21, 'Fear': 0.44}
Blood on the Leaves {'Happy': 0.11, 'Angry': 0.0, 'Surprise': 0.32, 'Sad': 0.29, 'Fear': 0.29}
Heartless {'Happy': 0.04, 'Angry': 0.04, 'Surprise': 0.28, 'Sad': 0.51, 'Fear': 0.12}
Love Lockdown {'Happy': 0.39, 'Angry': 0.03, 'Surprise': 0.06, 'Sad': 0.39, 'Fear': 0.13}
Touch the Sky {'Happy': 0.12, 'Angry': 0.02, 'Surprise': 0.34, 'Sad': 0.27, 'Fear': 0.25}


## Sentiment scores

In [ ]:
df_bs_0 = search_data('Maximillian', 1, client_access)
df_bs = clean_lyrics(df_bs_0,'lyric')
df_bs.to_csv('lyrics.csv',index=False)
df_bs.head(10)


In [6]:
# on sight
os = {}
os[0] = """Yeezy season approaching
Fuck whatever y'all been hearing
Fuck what, fuck whatever y'all been wearing
A monster about to come alive again
Soon as I pull up and park the Benz
We get this bitch shaking like Parkinsons
Take my number and lock it in
Indian hair, no moccasins
It's too many hoes in this house of sin
Real nigga back in the house again
Black Timbs all on your couch again
Black dick all in your spouse again
And I know she like chocolate men
She got more niggas off than Cochran, huh?

Baby girl tryna get a nut
And her girl tryna give it up
Chopped em both down
Don't judge 'em, Joe Brown
One last announcement
No sports bra, let's keep it bouncing
Everybody wanna live at the top of the mountain
Took her to the 'Bleau, she tried to sip the fountain
That when David Grutman kicked her out
But I got her back in and put my dick in her mouth
"""

for i in range(0):
  scores = te.get_emotion(os[i])
  print(i, scores)

0 {'Happy': 0.12, 'Angry': 0.06, 'Surprise': 0.06, 'Sad': 0.5, 'Fear': 0.25}
